# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/teradata_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Body"]]
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

617


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['oI have a teradata table with about 10 million records in it, that stores a '
 'numeric id field as a varchar. i need to transfer the values in this field '
 'to a bigint column in another table, but i cant simply say cast(id_field as '
 'bigint) because i get an invalid character error. looking through the '
 'values, i find that there could be a character at any position in the '
 'string, so lets say the string is varchar(18) i could filter out invalid '
 'rows like so : where substr(id_field,1,1) not in (/*big,ugly array of '
 'non-numeric chars*/) and substr(id_field,2,1) not in (/*big,ugly array of '
 'non-numeric chars*/) etc, etc... then the cast would work, but this is not '
 'feasible in the long run. its slow and if the string has 18 possible '
 'characters, it makes the query unreadable. how can i filter out rows that '
 'have a value in this field that will not cast as a bigint without checking '
 'each character individually for an array of non-numeric characters? examp

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['oi', 'have', 'teradata', 'table', 'with', 'about', 'million', 'records', 'in', 'it', 'that', 'stores', 'numeric', 'id', 'field', 'as', 'varchar', 'need', 'to', 'transfer', 'the', 'values', 'in', 'this', 'field', 'to', 'bigint', 'column', 'in', 'another', 'table', 'but', 'cant', 'simply', 'say', 'cast', 'id_field', 'as', 'bigint', 'because', 'get', 'an', 'invalid', 'character', 'error', 'looking', 'through', 'the', 'values', 'find', 'that', 'there', 'could', 'be', 'character', 'at', 'any', 'position', 'in', 'the', 'string', 'so', 'lets', 'say', 'the', 'string', 'is', 'varchar', 'could', 'filter', 'out', 'invalid', 'rows', 'like', 'so', 'where', 'substr', 'id_field', 'not', 'in', 'big', 'ugly', 'array', 'of', 'non', 'numeric', 'chars', 'and', 'substr', 'id_field', 'not', 'in', 'big', 'ugly', 'array', 'of', 'non', 'numeric', 'chars', 'etc', 'etc', 'then', 'the', 'cast', 'would', 'work', 'but', 'this', 'is', 'not', 'feasible', 'in', 'the', 'long', 'run', 'its', 'slow', 'and', 'if', 'the

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['teradata', 'table', 'record', 'store', 'numeric', 'field', 'varchar', 'need', 'transfer', 'value', 'field', 'bigint', 'column', 'table', 'simply', 'say', 'cast', 'bigint', 'get', 'invalid', 'character', 'error', 'look', 'value', 'find', 'character', 'position', 'string', 'let', 'say', 'string', 'filter', 'invalid', 'row', 'substr', 'big', 'ugly', 'array', 'non', 'numeric', 'char', 'substr', 'big', 'ugly', 'array', 'non', 'numeric', 'char', 'cast', 'work', 'feasible', 'long', 'run', 'slow', 'string', 'possible', 'character', 'make', 'query', 'unreadable', 'filter', 'row', 'value', 'field', 'cast', 'bigint', 'check', 'character', 'individually', 'array', 'non', 'numeric', 'character', 'example', 'value', 'abc', 'value', 'follow', 'specific', 'pattern', 'simply', 'need', 'filter', 'one', 'contain', 'non', 'numeric', 'datum', 'abc_c']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 3), (3, 2), (4, 3), (5, 3), (6, 2), (7, 5), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 4), (28, 5), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 2), (38, 2), (39, 1), (40, 1), (41, 1), (42, 3), (43, 2), (44, 2), (45, 1), (46, 1), (47, 2), (48, 1), (49, 5), (50, 1), (51, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abc', 1),
  ('abc_c', 1),
  ('array', 3),
  ('big', 2),
  ('bigint', 3),
  ('cast', 3),
  ('char', 2),
  ('character', 5),
  ('check', 1),
  ('column', 1),
  ('contain', 1),
  ('datum', 1),
  ('error', 1),
  ('example', 1),
  ('feasible', 1),
  ('field', 3),
  ('filter', 3),
  ('find', 1),
  ('follow', 1),
  ('get', 1),
  ('individually', 1),
  ('invalid', 2),
  ('let', 1),
  ('long', 1),
  ('look', 1),
  ('make', 1),
  ('need', 2),
  ('non', 4),
  ('numeric', 5),
  ('one', 1),
  ('pattern', 1),
  ('position', 1),
  ('possible', 1),
  ('query', 1),
  ('record', 1),
  ('row', 2),
  ('run', 1),
  ('say', 2),
  ('simply', 2),
  ('slow', 1),
  ('specific', 1),
  ('store', 1),
  ('string', 3),
  ('substr', 2),
  ('table', 2),
  ('teradata', 1),
  ('transfer', 1),
  ('ugly', 2),
  ('unreadable', 1),
  ('value', 5),
  ('varchar', 1),
  ('work', 1)]]

In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [11]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

100%|██████████| 540/540 [09:45<00:00,  1.08s/it]


In [12]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.997446755545206

Coherence Score:  0.593419664072752


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.133219 -0.119977       1        1  45.827457
1     -0.115152  0.091148       2        1  26.013704
0      0.015106  0.084096       3        1  22.064920
2      0.108670 -0.008052       4        1   3.155906
4      0.124595 -0.047215       5        1   2.938014, topic_info=         Term         Freq        Total Category  logprob  loglift
169      java   301.000000   301.000000  Default  30.0000  30.0000
158       com   236.000000   236.000000  Default  29.0000  29.0000
44      table   872.000000   872.000000  Default  28.0000  28.0000
45   teradata  1284.000000  1284.000000  Default  27.0000  27.0000
170      jdbc   180.000000   180.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
415     count     5.837772    42.920380   Topic5  -5.4594   1.5324
312    single     5.613441    52.339844   Topic5  -5.4986   1.2948
246    second     5.595149    53.998292   Topic5  -5.5019   1.2604
264    change     3.920446    44.281574   Topic5  -5.8576   1.1031
245    result     4.693263   129.705252   Topic5  -5.6776   0.2083

[259 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
156       1  0.049577     action
156       2  0.049577     action
156       3  0.892390     action
298       1  0.350666  aggregate
298       2  0.070133  aggregate
...     ...       ...        ...
51        2  0.194948       work
51        3  0.074539       work
640       1  0.963329       year
640       2  0.017839       year
640       3  0.017839       year

[715 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 5])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.029*"java" + 0.023*"teradata" + 0.023*"com" + 0.017*"jdbc" + '
  '0.013*"error" + 0.011*"sqoop" + 0.011*"spark" + 0.011*"run" + '
  '0.010*"option" + 0.008*"org_apache"'),
 (1,
  '0.047*"teradata" + 0.022*"error" + 0.019*"connect" + 0.016*"database" + '
  '0.015*"odbc" + 0.015*"import" + 0.015*"driver" + 0.013*"connection" + '
  '0.013*"use" + 0.013*"file"'),
 (2,
  '0.008*"office" + 0.006*"regexp_matche" + 0.005*"coalesce" + '
  '0.004*"lookbehind" + 0.004*"gi" + 0.004*"cli" + 0.004*"pointer_target" + '
  '0.004*"type_enable" + 0.004*"warning_return" + 0.004*"extraction"'),
 (3,
  '0.038*"table" + 0.022*"teradata" + 0.017*"error" + 0.016*"use" + '
  '0.015*"select" + 0.015*"column" + 0.015*"datum" + 0.014*"try" + '
  '0.011*"query" + 0.010*"date"'),
 (4,
  '0.039*"partition" + 0.009*"range" + 0.009*"range_n" + 0.009*"elimination" + '
  '0.008*"step" + 0.008*"amp" + 0.007*"unknown" + 0.006*"sum" + '
  '0.006*"explain" + 0.005*"condition"')]


In [12]:
num_topics = 5
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicTeradata.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

        Id                                              Title  \
0  3559698  how do i filter out non-numeric values in a te...   
1  3598327  Problem reading special characters from terada...   
2  4751146     Oracle XE Database link to Teradata using ODBC   
3  6017422  Threads hanging permanently on JDBC Teradata r...   
4  6110652       Executing teradata stored procedure from SAS   

                                                Body  RatingsSentiCR  \
0  oI have a teradata table with about 10 million...              -1   
1  I use teradata and the below query outputs "Al...              -1   
2  I installed Oracle Server Express 10g on my co...               1   
3  I'm using JDBC to query a Teradata server. The...               1   
4  I'm trying to execute Teradata stored procedur...               1   

  RatingsGPT35  RatingsGPTFineTuned  \
0           -1                 -1.0   
1           -1                 -1.0   
2           -1                 -1.0   
3           -1      